In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import javalang
from dataclasses import dataclass
from tqdm.auto import tqdm
from pickle_cache import PickleCache
from javalang import tree
from pprint import pprint
import textwrap
import copy
from iterextras import par_for, unzip
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from utils import *
sns.set()

In [3]:
solutions = get_solutions()

In [ ]:
# import traceback

# inlined = {}    
# fail = 0
# for student, solns in tqdm(list(solutions.items())):
#     try:
#         methods = get_methods(solns[-1])
#         start = copy.deepcopy(methods['run'])
#         inlined[student] = Inline(methods).visit(start).body
# #     except RecursionError:
# #         pass
#     except Exception:
#         #traceback.print_exc()
#         fail += 1

In [4]:
grammars = {}
for k, solns in tqdm(list(solutions.items())):   
    try:
        methods = get_methods(solns[-1])
        if 'run' not in methods:
            continue

        generator = GrammarGenerator(k, methods)
        generator.generate(methods['run'])
        grammars[k] = Grammar(generator.productions)
    except (RecursionError, Unimplemented, IndexError):
        pass
    except Exception:
        print(k)
        raise

In [6]:
g = Grammar(productions={k: p for g in grammars.values() for k, p in g.productions.items()})
g, cs = g.simplify()

In [14]:
groups = sorted(cs.items(), key=lambda t: len(t[1]))[::-1]
for k, group in groups[:50]:
    other_names = [k2 for k2 in group if '_rule' not in k2]
    print(k, len(group), other_names)
    print(g.productions[k])
    print()

student031_rule1 383 ['student593_moveUp']
Production(rule=[move()])

student324_rule0 315 ['student156_makeColumn', 'student287_putBeeperForward']
Production(rule=[move(), putBeeper()])

student324_rule1 241 ['student827_checkToMove', 'student118_alternateBeepers', 'student514_beeperEveryOtherCorner', 'student084_placeBeeperAndMove', 'student022_alternatingBeepers', 'student598_placeSingleBeeper']
Production(rule=[move(), if (frontIsClear()) { 'student324_rule0' }])

student031_rule0 119 []
Production(rule=[turnLeft()])

student031_rule2 110 []
Production(rule=[putBeeper()])

student677_rule4 83 []
Production(rule=[move(), putBeeper(), if (frontIsClear()) { 'student031_rule1' }])

student058_fillOddRow 81 ['student106_oddRow', 'student304_checkerOddStreet', 'student010_dropBeeper', 'student239_doOddRow', 'student599_layOddRow', 'student330_fillRow', 'student391_createFirstRow', 'student307_doOneRow', 'student134_fillAlphaRowForward', 'student187_leftToRight', 'student058_fillOddRow', 

In [ ]:
it = list(grammars.items())
fk, fg = it[0]
norm_grammars = {fk: fg}
for k1, g1 in tqdm(it[1:]):
    for k2, g2 in norm_grammars.items():
        g1, subs = g2.normalize(g1)
    norm_grammars[k1] = g1

In [ ]:
N = len(grammars)
dists = np.zeros((N, N))
index = list(grammars.keys())
rev_index = {s: i for i, s in enumerate(index)}

for k1, grammar1 in tqdm(list(grammars.items())):
    for k2, grammar2 in grammars.items():
        count = 0
        grammar1, grammar2 = normalize_grammars(grammar1, grammar2)
        for p1 in grammar1.productions.values():
            for p2 in grammar2.productions.values():
                if p1 == p2:
                    count += 1
                    break
        dists[rev_index[k1],rev_index[k2]] = count

In [ ]:
dists_sorted = np.dstack(np.unravel_index(np.argsort(dists, axis=None), dists.shape))[0,::-1]
dists_sorted = dists_sorted[dists_sorted[:,0] != dists_sorted[:,1]]

In [ ]:
[i, j] = dists_sorted[1]

In [ ]:
grammars[index[i]].productions

In [ ]:
grammars[index[j]].productions

In [ ]:
print(solutions[index[i]][-1].source)

In [ ]:
print(solutions[index[j]][-1].source)